<a href="https://colab.research.google.com/github/leomay282/notebooks_sprint5/blob/dev/Embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk                         # NLP toolbox
from os import getcwd
import pandas as pd                 # Library for Dataframes
from nltk.corpus import twitter_samples
import matplotlib.pyplot as plt     # Library for visualization
import numpy as np

nltk.download('twitter_samples')

[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Unzipping corpora/twitter_samples.zip.


True

In [ ]:
# select the set of positive and negative tweets
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

In [ ]:
all_positive_tweets[0]

'#FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)'

In [ ]:
all_negative_tweets[0]

'hopeless for tmr :('

In [ ]:
tweets = all_positive_tweets + all_negative_tweets ## Concatenate the lists.
labels = np.append(np.ones((len(all_positive_tweets),1)), np.zeros((len(all_negative_tweets),1)), axis = 0)

In [ ]:
tweets[:10]

['#FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)',
 '@Lamb2ja Hey James! How odd :/ Please call our Contact Centre on 02392441234 and we will be able to assist you :) Many thanks!',
 '@DespiteOfficial we had a listen last night :) As You Bleed is an amazing track. When are you in Scotland?!',
 '@97sides CONGRATS :)',
 'yeaaaah yippppy!!!  my accnt verified rqst has succeed got a blue tick mark on my fb profile :) in 15 days',
 '@BhaktisBanter @PallaviRuhail This one is irresistible :)\n#FlipkartFashionFriday http://t.co/EbZ0L2VENM',
 "We don't like to keep our lovely customers waiting for long! We hope you enjoy! Happy Friday! - LWWF :) https://t.co/smyYriipxI",
 '@Impatientraider On second thought, there’s just not enough time for a DD :) But new shorts entering system. Sheep must be buying.',
 'Jgh , but we have to go to Bayan :D bye',
 'As an act of mischievousness, am calling the ETL layer of our in-house warehousing 

In [ ]:
labels[:10]

array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.]])

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(tweets, labels,  test_size=0.2, shuffle=True, random_state=30)

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import re
import string

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer

def process_tweet(tweet):
    '''
    Input:
        tweet: a string containing a tweet
    Output:
        tweets_clean: a list of words containing the processed tweet

    '''
    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')
    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
    # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    # tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)

    tweets_clean = []
    for word in tweet_tokens:
        if (word not in stopwords_english and  # remove stopwords
            word not in string.punctuation):  # remove punctuation
            # tweets_clean.append(word)
            stem_word = stemmer.stem(word)  # stemming word
            tweets_clean.append(stem_word)

    return tweets_clean

In [ ]:
# Conjunto de palabras de mi conjunto de training limpio.// Se podria hacer para todo el corpus sin limpiar (para definir mi propia lista de stopwords/para saber la distribucion de palabras, etc)
train_words = []
for tweet in X_train:
  clean_tweet = process_tweet(tweet)
  train_words.extend(clean_tweet)

In [ ]:
# Con esto creamos el diccionario pero con las palabras agrupadas y contadas
from collections import Counter

In [ ]:
train_vocab_freq = Counter(train_words)
train_vocab_freq
# con NLTK?wordfrecuency puedo hacer lo mismo

Counter({'somi': 1,
         'definit': 22,
         'much': 174,
         'beauti': 47,
         'better': 74,
         'ever': 45,
         'guess': 22,
         'peopl': 99,
         'jealou': 10,
         'bc': 48,
         "ain't": 9,
         ':-)': 551,
         'home': 76,
         'time': 227,
         'today': 163,
         ':(': 3649,
         'dublin': 1,
         'chalut': 1,
         ':)': 2866,
         'readi': 29,
         'poldi': 1,
         'cimbom': 1,
         'look': 189,
         'fun': 76,
         'snapchat': 85,
         'jennyjean': 2,
         '22': 5,
         'hornykik': 15,
         'eboni': 1,
         'trade': 4,
         'model': 11,
         'elfindelmundo': 2,
         'webcamsex': 4,
         'sorri': 136,
         'closest': 3,
         'door': 7,
         'morn': 104,
         'thank': 581,
         'join': 27,
         'us': 123,
         'happi': 189,
         'friday': 99,
         'mommi': 3,
         'left': 31,
         'hey': 86,
         

In [ ]:
#Ordeno de acuerdo a la pa;abra que mas se repite/ ESTOY DEBERIA HACERSE ANTES DE TODO LO QUE HICIMOS.
#pero aqui puedo quitar palabras a simbolos, segun experimientacion. ahora lo dejamos, entreno el modelo y mas adelante podria entrenar otro modelo quitandolo.

sorted(train_vocab_freq.items(), key=lambda x:x[1], reverse=True)[:10]

[(':(', 3649),
 (':)', 2866),
 ('thank', 581),
 (':-)', 551),
 ('follow', 518),
 (':d', 495),
 ('...', 491),
 ('love', 437),
 ("i'm", 419),
 (':-(', 394)]

In [ ]:
# Tengo un array de arrays donde cada lista ya esta tokenizada, esta limpio.
X_train_clean = []
for tweet in X_train:
  clean_tweet = process_tweet(tweet)
  X_train_clean.append(clean_tweet)

In [ ]:
X_train_clean[:5]

[['somi',
  'definit',
  'much',
  'beauti',
  'better',
  'ever',
  'guess',
  'peopl',
  'jealou',
  'bc',
  "ain't",
  ':-)'],
 ['home', 'time', 'today', ':(', 'dublin'],
 ['chalut', ':)'],
 ['readi', ':)', 'poldi', 'cimbom'],
 ['look',
  'fun',
  'snapchat',
  'jennyjean',
  '22',
  'snapchat',
  'hornykik',
  'eboni',
  'trade',
  'model',
  'elfindelmundo',
  'webcamsex',
  ':(']]

In [ ]:
# lo que eliminamos...ademas de lo que tenemos en la funcion
stopword_english = stopwords.words('english')
stopword_english

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

# TF-IDF approach

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# la libreria no me acepta el texto tokenizado, le voy a pasar la funcion.
tf_idf = TfidfVectorizer(analyzer='word', tokenizer=process_tweet)
# agarra cada palabra y luego se va por el documento contando en cuantos aparece cada palabra etc...ahi saca idf
tf_idf.fit(X_train) # IDF - Inverse Document Frequency ----- genera los pesos de cada palabra para que luego yo pueda hacer una transformacion // Hace un conteo interno de manera que luego me permita transformar.

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


TfidfVectorizer(tokenizer=<function process_tweet at 0x7ef528f20ca0>)

In [ ]:
X_train[10]

'@inespsousa9 not me! :('

In [ ]:
# ahora transformo los datos de train a una representacion numerica // ahora saca la frecuencia de cada palabtra en cada doc y la multiplica por idf de esa palabra
# y contruye un version numerica del tweet
training_vectors = tf_idf.transform(X_train)

In [ ]:
# no me la muestra porqye es una matriz dispersa, puede tener muchos ceros.
training_vectors

<8000x9088 sparse matrix of type '<class 'numpy.float64'>'
	with 51768 stored elements in Compressed Sparse Row format>

In [ ]:
X_train[10]

'@inespsousa9 not me! :('

In [ ]:
#El tweet anterior solo tiene 4 tokken de los 9088 del vocabulario (0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5.656,0,0,0,0,0,0,0,0,0...) matrix sparse
len(tf_idf.vocabulary_)

9088

# K-NN Model

In [ ]:
from sklearn.neighbors import NearestNeighbors

In [ ]:
nbrs = NearestNeighbors(n_neighbors=3).fit(training_vectors)

# Logistic Regression

In [ ]:
# para dos clases se sabe usar una regresion logistica
from sklearn.linear_model import LogisticRegression

In [ ]:
clf = LogisticRegression(C=1, tol=0.1)

In [ ]:
clf.fit(training_vectors, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1, tol=0.1)

In [ ]:
# debemos usar los datos de test, pero antes debemos vectorizarlos..para uso practico, solo usamos el data de entrenamiento.
y_train_pred = clf.predict(training_vectors)

In [ ]:
clf.score(training_vectors, y_train)

0.999125

# SVM